In [1]:
import os

from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.table import Table, unique, vstack, join, Column

from sys import path
path.insert(0, '..')
from surface_brightness import surface_brightness


/Users/Daniel/anaconda/lib/python3.4/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [2]:
#Input / Output Information
input_data_table = 'observed targets data table.csv'
fits_directory = '/Users/Daniel/Downloads/Other Supernova/' #The directory to look for .fits files in
output_file_name = "observed targets" #The output file name without an extension

sn_col_name = 'Name'
redshift_col_name = 'z'
ra_col_name = 'RA'
dec_col_name = 'Dec'

In [4]:
if os.path.exists(fits_directory) and os.listdir(fits_directory):
    cord_dict = {} #for supernova coordinates
    red_dict = {}#for supernova redshifts
    
    observed_targets_table = Table.read(input_data_table, format = 'csv')
    #Read in the input data table
    for row in observed_targets_table:
        red_dict[row[sn_col_name]] = row[redshift_col_name]
        cord_dict[row[sn_col_name]] = SkyCoord(row[ra_col_name] + ' ' + row[dec_col_name], unit = (u.hourangle, u.deg))
    
    brightness = surface_brightness(cord_dict, red_dict)
    
    #Individual tables are generated for nuv and fuv using 1kpc and 5kpc apertures
    print('Calculating NUV values using a 1kpc radius aperture\n', flush = True)
    nuv_1kpc = brightness.create_tables('NUV', fits_directory, 1)
    
    print('Calculating FUV values using a 1kpc radius aperture\n', flush = True)
    fuv_1kpc = brightness.create_tables('FUV', fits_directory, 1)
    
    print('Calculating NUV values using a 5kpc radius aperture\n', flush = True)
    nuv_5kpc = brightness.create_tables('NUV', fits_directory, 5)
    
    print('Calculating FUV values using a 5kpc radius aperture\n', flush = True)
    fuv_5kpc = brightness.create_tables('FUV', fits_directory, 5)
    
    #Create plots and save them to file
    print('Creating Plots\n', flush = True)
    brightness.create_plots(nuv_1kpc[0], 'NUV', 1)
    brightness.create_plots(fuv_1kpc[0], 'FUV', 1)
    
    #Create a table with the desired output information
    print('Joining Tables\n', flush = True)
    nuv_1kpc[0].keep_columns(['sn',
                              'NUV 1kpc Luminosity',
                              'NUV 1kpc Luminosity Error',
                              'NUV 1kpc Surface Brightness',
                              'NUV 1kpc Surface Brightness Error'])
    
    fuv_1kpc[0].keep_columns(['sn',
                              'FUV 1kpc Luminosity',
                              'FUV 1kpc Luminosity Error',
                              'FUV 1kpc Surface Brightness',
                              'FUV 1kpc Surface Brightness Error'])
    
    nuv_5kpc[0].remove_columns(['Flux',
                                'Flux Error',
                                'NUV 5kpc Luminosity',
                                'NUV 5kpc Luminosity Error'])
    
    fuv_5kpc[0].remove_columns(['Flux',
                                'Flux Error',
                                'FUV 5kpc Luminosity',
                                'FUV 5kpc Luminosity Error'])
    
    merged_nuv_table = join(nuv_1kpc[0], nuv_5kpc[0], join_type = 'outer', keys = ['sn'])
    merged_fuv_table = join(fuv_1kpc[0], fuv_5kpc[0], join_type = 'outer', keys = ['sn'])
    
    merged_table = join(merged_nuv_table, merged_fuv_table, join_type = 'outer', keys = ['sn'])
    merged_table.rename_column('NUV 5kpc Exposure Time', 'NUV Exposure Time')
    merged_table.rename_column('FUV 5kpc Exposure Time', 'FUV Exposure Time')
    merged_table.rename_column('Redshift_1', 'Redshift')
    merged_table.rename_column('Redshift Error_1', 'Redshift Error')
    
    for row in merged_table:
        if str(row["Redshift"]) == "--":
            row["Redshift"] = row["Redshift_2"]
            row["Redshift Error"] = row["Redshift Error_2"]
    
    merged_table.remove_columns(["Redshift_2", "Redshift Error_2"])
    
    #Rearrange the column order
    final_table = merged_table['sn', 'Redshift', 'Redshift Error',
                               'NUV Exposure Time',
                               'NUV 1kpc Luminosity',
                               'NUV 1kpc Luminosity Error',
                               'NUV 1kpc Surface Brightness',
                               'NUV 1kpc Surface Brightness Error',
                               'NUV 5kpc Surface Brightness',
                               'NUV 5kpc Surface Brightness Error',
                               'FUV Exposure Time',
                               'FUV 1kpc Luminosity',
                               'FUV 1kpc Luminosity Error',
                               'FUV 1kpc Surface Brightness',
                               'FUV 1kpc Surface Brightness Error',
                               'FUV 5kpc Surface Brightness',
                               'FUV 5kpc Surface Brightness Error']
    
    final_table.sort('sn')
    
    merged_log = vstack([nuv_5kpc[1], fuv_5kpc[1]])
    merged_log.sort('File Path')
    
    #Write data to an output file
        #The csv format does not have an 'overwrite' kwarg,
        #but an existing file will be overwritten
    final_table.write(output_file_name + '.fits', overwrite = True, format = 'fits')
    final_table.write(output_file_name + '.csv', format = 'ascii.csv')
    merged_log.write(output_file_name + ' log.csv', format = 'ascii.csv')
    print('Finished')
    
elif not os.path.exists(fits_directory):
    print('Specified directory of .fits files does not exist')
    
else:
    print('Specified directory of .fits files is empty')


Calculating NUV values using a 1kpc radius aperture

Calculating FUV values using a 1kpc radius aperture

Calculating NUV values using a 5kpc radius aperture

Calculating FUV values using a 5kpc radius aperture

Creating Plots

Joining Tables



Finished
